# Robot Movement and Animation Demo

This notebook demonstrates how to create, manipulate, and animate robot movements using the LinkMotion library.

## Overview
This example showcases:
1. **Robot Construction**: Building a humanoid robot with multiple links and joints
2. **Joint Movement**: Moving individual joints and visualizing the results
3. **Animation**: Creating smooth animations of robot movements over time

## Key Components
- `Robot`: The main robot structure
- `Link`: Individual robot parts (body, arms, legs, head)
- `Joint`: Connections between links with movement constraints
- `MoveManager`: Handles robot joint movements
- `MoveVisualizer`: Renders robot states and animations

## Robot Structure
The humanoid robot includes:
- Body (box shape)
- Left and right arms (cylinders)
- Left and right legs (cones)
- Head (sphere)
- Left foot (capsule)
- Joints connecting body to left leg and left leg to foot

Let's start by importing the required libraries:

In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R

from linkmotion.visual import MoveVisualizer
from linkmotion import Robot, Link, Joint, JointType, Transform, MoveManager

## Robot Definition

Here we construct a complete humanoid robot with multiple links and joints:

In [ ]:
humanoid = Robot()

body = Link.from_box("body", extents=np.array((3, 2, 10)))

t = Transform(
    rotate=R.from_rotvec(90.0 * np.array((0, 1, 0)), degrees=True),
    translate=np.array((4, 0, 3)),
)
right_arm = Link.from_cylinder("right_arm", radius=0.5, height=5, default_transform=t)

t = Transform(
    rotate=R.from_rotvec(90.0 * np.array((0, 1, 0)), degrees=True),
    translate=np.array((-4, 0, 3)),
)
left_arm = Link.from_cylinder("left_arm", radius=0.5, height=5, default_transform=t)

t = Transform(
    rotate=R.from_rotvec(180.0 * np.array((0, 1, 0)), degrees=True),
    translate=np.array((1, 0, -8)),
)
right_leg = Link.from_cone("right_leg", radius=0.5, height=6, default_transform=t)

t = Transform(
    rotate=R.from_rotvec(180.0 * np.array((0, 1, 0)), degrees=True),
    translate=np.array((-1, 0, -8)),
)
left_leg = Link.from_cone("left_leg", radius=0.5, height=6, default_transform=t)

t = Transform(
    rotate=R.from_rotvec(180.0 * np.array((0, 1, 0)), degrees=True),
    translate=np.array((-1, 0, -10.5)),
)
left_foot = Link.from_capsule("left_foot", radius=0.5, height=0.2, default_transform=t)

t = Transform(translate=np.array((0, 0, -5)))
head = Link.from_sphere("head", 3, center=np.array((0, 0, 8)))

left_leg_joint = Joint(
    "left_leg_joint",
    JointType.REVOLUTE,
    child_link_name="left_leg",
    parent_link_name="body",
    center=np.array((-1, 0, -5)),
    direction=np.array((1, 0, 0)),
    min_=-np.pi / 2.0,
    max_=np.pi / 2.0,
)

left_foot_joint = Joint(
    "left_foot_joint",
    JointType.REVOLUTE,
    child_link_name="left_foot",
    parent_link_name="left_leg",
    center=np.array((-1, 0, -10.5)),
    direction=np.array((1, 0, 0)),
    min_=-np.pi / 2.0,
    max_=np.pi / 2.0,
)

humanoid.add_link(body)
humanoid.add_link(right_arm)
humanoid.add_link(left_arm)
humanoid.add_link(right_leg)
humanoid.add_link(left_leg)
humanoid.add_link(head)
humanoid.add_link(left_foot)
humanoid.add_joint(left_leg_joint)
humanoid.add_joint(left_foot_joint)

mm = MoveManager(humanoid)

## Single Joint Movement

This example shows how to move a single joint and visualize the robot's new configuration:

In [ ]:
mm.move(joint_name="left_leg_joint", value=-np.pi / 3.0)
plot = MoveVisualizer.robot(mm)
plot = MoveVisualizer.joint(mm, "left_leg_joint", plot=plot)
plot = MoveVisualizer.joint(mm, "left_foot_joint", plot=plot)
plot.display()

mm.reset_move()

## Output Example
![OutputExample](./img/move/move1.png)

## Animated Movement Sequence

This creates a smooth animation showing the robot's movement over time with multiple joint positions:

In [ ]:
command_series = {
    1.0: {"left_leg_joint": -np.pi / 12.0, "left_foot_joint": 0.0},
    2.0: {"left_leg_joint": -np.pi / 9.0, "left_foot_joint": 0.0},
    3.0: {"left_leg_joint": -np.pi / 6.0, "left_foot_joint": 0.0},
    4.0: {"left_leg_joint": -np.pi / 3.0, "left_foot_joint": 0.0},
    5.0: {"left_leg_joint": -np.pi / 2.0, "left_foot_joint": 0.0},
    6.0: {"left_leg_joint": -np.pi / 2.0, "left_foot_joint": -np.pi / 2.0},
}
link_names = {link.name for link in humanoid.links()}
MoveVisualizer.move(mm, command_series, set(link_names))
# ↓ Animation

## Output Example
![OutputExample](./img/move/move2.png)